# Extract Single Date by Polygons
This notebook is designed to iterate through a polygon shapefile and extract image chips based on the fields in the attribute table realting to capture date and sensor mission. This notebook will only extract a bounding rectangle for each shape (incuding multipart shapes rather than an exact polygon).

In [75]:
import datacube
import xarray as xr
from datacube.drivers.netcdf import write_dataset_to_netcdf
import geopandas as gpd
from datacube.utils import geometry
from datacube.utils.cog import write_cog

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import download_unzip
from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb
from dea_tools.spatial import xr_rasterize

In [76]:
dc = datacube.Datacube(app='Polygon_LUT_query')

In [77]:
# Input LUT polygon file
LUT_polygon = '/home/jovyan/DPIPWE_input/DEA_multi__sen_shapes.shp'

In [78]:
# Read unzipped shapefile
gdf = gpd.read_file(LUT_polygon)

# Check that the polygon loaded as expected. We'll just print the first 3 rows to check
gdf.head(3)

,Id,SENSOR,SEN_DATE,NAME,geometry
0,0,SENTINEL,2019-11-16,Jilletts Tier,"POLYGON ((501065.355 5346765.939, 503843.485 5..."
1,0,LANDSAT,2019-01-29,Boobyalla,"POLYGON ((574145.622 5473413.415, 576295.366 5..."
2,0,SENTINEL,2019-01-30,Flinders_multi,"MULTIPOLYGON (((600923.053 5550302.147, 599600..."


In [79]:
# Initialise analysis parameters (not really necessary)
sdate = '' #start date
edate = '' #end date
name = '' #name attribute to label output 
product = '' #attribute value for product/sensor

#measurements = ['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_nir'] #assess how to handle Landsat v Sentinel
crs = gdf.crs
#align = (0, 0)

In [86]:
# Dictionary to ve results
results = {} 

# Loop through polygons in geodataframe (gdf) and extract satellite data
for index, row in gdf.iterrows():
    
    print(f'Feature: {index + 1}/{len(gdf)}')
    
    # Extract the feature's geometry as a datacube geometry object
    geom = geometry.Geometry(geom=row.geometry, crs=gdf.crs)
    sen = (row['SENSOR'])
    sdate = (row['SEN_DATE'])
    name = (row['NAME'])
    
    if sen == 'SENTINEL':
        # Load available data from both Sentinel 2 satellites
        products=['s2a_ard_granule', 's2b_ard_granule']
        measurements=['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_1'] #For 10m false colour
        resolution = (-10, 10) #For 10m false colour
        #measurements=['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_swir_2']
        #resolution = (-20, 20)
                
    elif sen == 'LANDSAT':
        products=['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3'],
        measurements=['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']
        resolution = (-30, 30)
        
    else:
        print('Sensor not recognised. No data loaded')
    
    #print(crs)
    query = {'geopolygon': geom,
         'time': sdate,
         'measurements': measurements,
         'resolution': resolution,
         'output_crs':crs
         }
    #print(query)
    
    # Load data (can also use 'load_ard' (will mask clouds))
    ds = load_ard(dc=dc, 
              products=products,
              # min_gooddata=0.99,  # only take uncloudy scenes
              mask_pixel_quality=False, # change for cloud masking
              #ls7_slc_off = False, # Default takes slc
              group_by= 'solar_day',
              **query)
    ''' 
    # Can use dc.load to load derived products
    ds = dc.load(product=products, group_by= 'solar_day', **query)
    '''
    # Mask the dataset to our polygon boundary
    # Create a mask of the indexed polygon
    mask = xr_rasterize(gdf.iloc[[index]], ds)
    
    # Mask dataset to set pixels outside the polygon to `NaN`
    ds_masked = ds.where(mask)
    
    # Append results to a dictionary using the name
    # column as an key
    results.update({name: ds_masked})

Feature: 1/3
Finding datasets
    s2a_ard_granule
    s2b_ard_granule
Loading 1 time steps
Feature: 2/3
Finding datasets
    ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
Loading 1 time steps
Feature: 3/3
Finding datasets
    s2a_ard_granule
    s2b_ard_granule
Loading 1 time steps


/env/lib/python3.8/site-packages/datacube/utils/geometry/_base.py:608: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return type(geom)([segmentize_shapely(g) for g in geom])
/env/lib/python3.8/site-packages/datacube/utils/geometry/_base.py:608: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return type(geom)([segmentize_shapely(g) for g in geom])


In [87]:
# Loop through dictionary and export to COG
for key, value in results.items():    
    item_da = value.isel(time=0).to_array() # Need to convert to array
    write_cog(geo_im=item_da,
          fname= key+'.tif',
          overwrite=True)